In [207]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [208]:
#Importing Data

training_set_features=pd.read_csv(r"C:\Users\hp\Downloads\dataset and all\training_set_features.csv")
training_set_labels=pd.read_csv(r"C:\Users\hp\Downloads\dataset and all\training_set_labels.csv")
test_set_features=pd.read_csv(r"C:\Users\hp\Downloads\dataset and all\test_set_features.csv")
#test_set_labels=pd.read_csv(r"C:\Users\hp\Downloads\dataset and all\submission_format.csv")

In [209]:
#training dataset(along with 2 labels)
df=pd.concat((training_set_features,training_set_labels))
df.set_index('respondent_id',inplace=True)

# test data set with features only
test_set_features.set_index('respondent_id',inplace=True)

In [210]:
#Numerical Columns in training and test data

numerical_columns= df.select_dtypes('number').columns
numerical_columns_test=test_set_features.select_dtypes('number').columns

In [211]:
#Filling NaN values in Numerical columns with respective median values

for col in numerical_columns:
    df[col].fillna(df[col].median(),inplace=True)
    
for col in numerical_columns_test:
    test_set_features[col].fillna(test_set_features[col].median(),inplace=True)

In [212]:
#Categorical Columns in training and test data

categorical_columns=df.select_dtypes('object').columns
categorical_columns_test=test_set_features.select_dtypes('object').columns

In [213]:
#Filling NaN values in categorical columns with None

for col in categorical_columns:
    df[col].fillna(value='None',inplace=True)

for col in categorical_columns_test:
    test_set_features[col].fillna(value='None',inplace=True)

In [215]:
#Encoding Label to categorical data

from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df[['age_group','education','race','sex','income_poverty','marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa'
               ,'employment_industry','employment_occupation']] = df[['age_group','education','race','sex','income_poverty','marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa'
               ,'employment_industry','employment_occupation']].apply(labelencoder.fit_transform)

test_set_features[['age_group','education','race','sex','income_poverty','marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa'
               ,'employment_industry','employment_occupation']] = test_set_features[['age_group','education','race','sex','income_poverty','marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa'
               ,'employment_industry','employment_occupation']].apply(labelencoder.fit_transform)



In [216]:
x=df.drop(['xyz_vaccine','seasonal_vaccine'],axis=1)
y=df[['xyz_vaccine','seasonal_vaccine']]

In [217]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.05, random_state=100) #Using some of the data for validation
X_train.shape,X_test.shape

((50743, 35), (2671, 35))

In [218]:
#Standardizing train and test dataset

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
test_set_features_scaled=scaler.transform(test_set_features)

In [219]:
!pip install scikit-multilearn

In [220]:
from skmultilearn.problem_transform import BinaryRelevance

In [221]:
from sklearn.naive_bayes import GaussianNB
model_naivebayes=BinaryRelevance(GaussianNB())
model_naivebayes.fit(X_train_scaled,y_train)

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [222]:
prediction_naivebayes=model_naivebayes.predict_proba(X_test_scaled)

In [223]:
from sklearn.metrics import accuracy_score

In [224]:
from sklearn.metrics import roc_auc_score

In [225]:
prediction_naivebayes=prediction_naivebayes.toarray()

naivebayes_auc=roc_auc_score(y_test,prediction_naivebayes,average='macro')
naivebayes_auc

0.7988169895009635

In [226]:
from sklearn.linear_model import LogisticRegression
model_logisticRegression = BinaryRelevance(LogisticRegression())
model_logisticRegression.fit(X_train_scaled,y_train)

BinaryRelevance(classifier=LogisticRegression(), require_dense=[True, True])

In [227]:
prediction_logisticRegression = model_logisticRegression.predict_proba(X_test_scaled)

In [228]:
prediction_logisticRegression=prediction_logisticRegression.toarray()

logisticregression_auc=roc_auc_score(y_test,prediction_logisticRegression,average='macro')
logisticregression_auc

0.7988169895009635

#Getting Same auc_score using both logistic regression and naive nayes

# Creating Final submission file Using the Logistic regression model

In [190]:
submission= model_logisticRegression.predict_proba(test_set_features_scaled)
submission=submission.toarray()
submission

array([[6.71962333e-06, 5.29021335e-06],
       [7.90815401e-06, 6.95224448e-06],
       [7.82366212e-06, 7.51067002e-06],
       ...,
       [2.70279014e-07, 1.32678287e-07],
       [8.15495512e-06, 6.94484304e-06],
       [6.64720118e-06, 4.77285819e-06]])

In [229]:
submission=pd.DataFrame(submission,columns=['h1v1_vaccine','seasonal_vaccine'])
submission

,h1v1_vaccine,seasonal_vaccine
respondent_id,,
26707,6.719623e-06,5.290213e-06
26708,7.908154e-06,6.952244e-06
26709,7.823662e-06,7.510670e-06
26710,2.060014e-07,1.115154e-07
26711,2.440330e-06,1.942101e-06
...,...,...
53410,8.884507e-05,8.262144e-05
53411,4.917638e-07,3.522281e-07
53412,2.702790e-07,1.326783e-07


In [230]:
submission.set_index(test_set_features.index,inplace=True)
submission


,h1v1_vaccine,seasonal_vaccine
respondent_id,,
26707,6.719623e-06,5.290213e-06
26708,7.908154e-06,6.952244e-06
26709,7.823662e-06,7.510670e-06
26710,2.060014e-07,1.115154e-07
26711,2.440330e-06,1.942101e-06
...,...,...
53410,8.884507e-05,8.262144e-05
53411,4.917638e-07,3.522281e-07
53412,2.702790e-07,1.326783e-07


In [232]:
submission.to_csv(r"C:\Users\hp\downloads")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\hp\\downloads'